In [30]:
import numpy as np
import pandas as pd
from scipy.stats import norm, multivariate_normal
import time

data = pd.read_csv('pulsar_stars.csv') # Source: R. J. Lyon, HTRU2, DOI: 10.6084/m9.figshare.3080389.v1. 

# 16,000 points from the original file are used as training data.
# The remaining data are used as test data. 

train_data = data.iloc[:16000,:8]
train_labels = data.iloc[:16000,8]

test_data = data.iloc[16000:, :8]
test_labels = data.iloc[16000:, 8]


In [28]:
# Input: training data (feature values only), training labels (corresponding class values only)
def model(x,y):
    x = np.array(x)
    y = np.array(y)
    num_features = 8
    
    mean = np.zeros((2, num_features)) # Vector containing mean of each feature for each class (pulsar, not a pulsar)
    cov_matrix = np.zeros((2, num_features, num_features)) # Covariance matrices for each class
    pi = np.zeros(2) # Weights for each class
    
    for status in range(0,2): # For each class
        indices = (y == status) # Array of 0s and 1s keeping track of what indices the class occurs at in the data
        pi[status] = float(sum(indices))/float(len(y)) # Calculates the weight for the class (# occurences of class / total # of candidates)
        mean[status] = np.mean(x[indices,:], axis=0) # Calculates mean of every feature in the class
        cov_matrix[status] = np.cov(x[indices,:], rowvar=0, bias=1) # Fills covariance matrix for the class
    
    return mean, cov_matrix, pi


means, covariance, weights = model(train_data, train_labels) # All the parameters we need to fit a Gaussian to each class

In [29]:
def predict(testx, testy):
    testx = np.array(testx)
    testy = np.array(testy)
    num_vectors = len(testy)
    score = np.zeros((num_vectors, 2))
    
    # Predict the class with maximum probability according to Bayes rule
    for i in range(0, num_vectors):
        for status in range(0,2):
            score[i, status] = np.log(weights[status]) + \
            multivariate_normal.logpdf(testx[i, range(0,8)], mean=means[status, range(0,8)], cov=covariance[status, range(0,8), range(0,8)])
    
    predictions = np.argmax(score, axis=1)
    
    errors = np.sum(predictions != testy)
    perror = errors/num_vectors * 100

    print("Classification error: " + str(round(perror, 2)) + "%")

t_before = time.time()
predict(test_data, test_labels)
t_after = time.time()

print("Run time: ", t_after - t_before, "s")
    
    

Classification error: 3.11%
Run time:  1.0086126327514648 s
